# 案例 1.6: 系统辨识与参数自动校准

本案例演示了平台的参数辨识（或称“自动校准”）能力。一个`ParameterIdentificationAgent`智能体将通过观测降雨和径流数据，来自动校准一个`RainfallRunoff`（降雨-径流）模型中的关键参数——径流系数。

### 1. 导入库并加载配置

In [ ]:
import json
import numpy as np
from swp.central_coordination.collaboration.message_bus import MessageBus
from swp.simulation_identification.physical_objects.rainfall_runoff import RainfallRunoff
from swp.simulation_identification.identification.identification_agent import ParameterIdentificationAgent

CONFIG_PATH = 'examples/parameter_identification.json'

with open(CONFIG_PATH, 'r') as f:
    config = json.load(f)

print("配置加载成功！")

### 2. 定义仿真函数

该函数将执行以下步骤：
1.  根据配置中的“真实”参数，生成一组模拟的降雨和“观测”径流数据。
2.  创建一个带有错误初始参数的`RainfallRunoff`模型实例。
3.  配置`ParameterIdentificationAgent`，使其监听数据并校准上述模型。
4.  模拟数据流，触发智能体的校准流程。
5.  输出并验证校准结果。

In [ ]:
def run_identification_example(cfg):
    # 1. 定义真实参数并生成合成数据
    true_params = cfg['ground_truth']
    data_gen_cfg = cfg['data_generation']
    num_points = data_gen_cfg['num_points']
    
    rainfall_series = np.random.rand(num_points) * data_gen_cfg['max_rainfall_intensity']
    observed_runoff = true_params['runoff_coefficient'] * rainfall_series * true_params['catchment_area']

    print(f"生成了 {num_points} 组降雨和径流的模拟观测数据。")
    print(f"真实的径流系数为: {true_params['runoff_coefficient']}\n")

    # 2. 创建一个带有错误初始参数的模型
    initial_guess = cfg['model_initial_guess']
    initial_params = {
        'catchment_area': true_params['catchment_area'], # 假设面积是已知的
        'runoff_coefficient': initial_guess['runoff_coefficient']
    }
    model_to_identify = RainfallRunoff(name="Catchment_A", parameters=initial_params)
    print(f"模型创建时使用的初始(错误)径流系数为: {initial_params['runoff_coefficient']}\n")

    # 3. 设置辨识智能体
    bus = MessageBus()
    agent_cfg = cfg['agent_config']
    agent = ParameterIdentificationAgent(
        agent_id="Identifier",
        target_model=model_to_identify,
        message_bus=bus,
        config=agent_cfg
    )

    # 4. 模拟数据采集过程
    print("开始模拟数据流，通过消息总线发布数据...")
    data_map = agent_cfg['data_map']
    for i in range(num_points):
        bus.publish(data_map['rainfall'], {'value': rainfall_series[i]})
        bus.publish(data_map['observed_runoff'], {'value': observed_runoff[i]})

    # 5. 触发辨识过程
    agent.run(current_time=0)

    # 6. 验证结果
    final_params = model_to_identify.get_parameters()
    final_coeff = final_params['runoff_coefficient']
    print(f"\n模型校准后的最终径流系数为: {final_coeff:.4f}")

    if abs(final_coeff - true_params['runoff_coefficient']) < 1e-3:
        print("成功: 辨识出的参数非常接近真实值。")
    else:
        print("注意: 辨识出的参数已朝着真实值优化。")

### 3. 运行仿真

In [ ]:
run_identification_example(config)